In [34]:
import time
import pandas as pd
from langchain_ollama import ChatOllama
from tqdm import tqdm
import subprocess
import psutil
import json

In [35]:
questions_df = pd.read_csv("questions.csv")
questions = questions_df["questions"].tolist()

In [36]:
models = ["qwen2.5:32b"]

In [37]:
def get_rocm_memory_usage():
    try:
        result = subprocess.run(["rocm-smi", "--showmemuse", "--json"], capture_output=True, text=True, check=True)
        data = json.loads(result.stdout)
        gpu_memory_usage = data["card0"]["GPU Memory Allocated (VRAM%)"]
        return int(gpu_memory_usage)
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

In [38]:
def test_model(model, questions, model_name):
    total_time = 0
    memory_usage = []

    columns = [
        "question", "answer", "time_taken", 
        "gpu_memory_before", "gpu_memory_after", "gpu_memory_delta"
    ]
    model_df = pd.DataFrame(columns=columns)

    model_df.to_csv(f"{model_name}_results.csv", mode='w', header=True, index=False)

    for question in tqdm(questions):
        start_time = time.time()

        gpu_used_before = get_rocm_memory_usage()

        print(question)
        response = model.invoke(question)
        print(response)

        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time

        gpu_used_after = get_rocm_memory_usage()

        result = {
            "question": question,
            "answer": response,
            "time_taken": elapsed_time,
            "gpu_memory_before": gpu_used_before,
            "gpu_memory_after": gpu_used_after,
            "gpu_memory_delta": gpu_used_after - gpu_used_before
        }
        
        # Используем pd.concat вместо .append
        model_df = pd.concat([model_df, pd.DataFrame([result])], ignore_index=True)
        
        model_df.to_csv(f"{model_name}_results.csv", mode='a', header=False, index=False)


        memory_usage.append({
            "time": elapsed_time,
            "gpu_memory_before": gpu_used_before,
            "gpu_memory_after": gpu_used_after,
            "gpu_memory_delta": gpu_used_after - gpu_used_before,
            "cpu_usage": psutil.cpu_percent(interval=1),  # Загрузка CPU
            "ram_usage": psutil.virtual_memory().percent  # Использование RAM
        })

    resource_df = pd.DataFrame(memory_usage)
    resource_df.to_csv(f"{model_name}_resources.csv", index=False)

    return total_time

In [39]:
total_time_all_models = {}
overall_times = []

for model in models:
    print(f"Тестирование модели: {model}")
    llm_model = ChatOllama(model=model)
    
    total_time = test_model(llm_model, questions, model)
    total_time_all_models[model] = total_time
    overall_times.append({
        "model": model,
        "total_time": total_time
    })
    print(f"Общее время для модели {model}: {total_time} секунд")

total_time_overall = sum(total_time_all_models.values())
print(f"Общее время на все вопросы для всех моделей: {total_time_overall} секунд")

overall_times_df = pd.DataFrame(overall_times)
overall_times_df.to_csv("overall_times.csv", index=False)

Тестирование модели: qwen2.5:32b


  0%|                                                                                                         | 0/50 [00:00<?, ?it/s]

Что такое искусственный интеллект?
content='Искусственный интеллект (ИИ) — это область науки и технологий, которая занимается созданием систем или программных приложений, способных имитировать человеческий интеллект для выполнения задач. Эти системы могут обучаться от новых данных (обучение с учителем и без учителя), использовать правила для достижения определенных целей (правила принятия решений) и самостоятельно принимать решения.\n\nИскусственный интеллект может быть представлен в различных формах, таких как речевые помощники (например, Siri или Alexa), программы машинного обучения, системы компьютерного зрения и др. В зависимости от способностей, ИИ можно разделить на слабый и сильный: слабый ИИ выполняет конкретные задачи (например, поиск информации в интернете), тогда как сильный ИИ может решать широкий спектр задач на уровне человека или даже превосходить его.' additional_kwargs={} response_metadata={'model': 'qwen2.5:32b', 'created_at': '2025-03-31T14:58:51.732527662Z', 'done':

/tmp/ipykernel_275694/4189172495.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_df = pd.concat([model_df, pd.DataFrame([result])], ignore_index=True)
  2%|█▉                                                                                               | 1/50 [00:18<15:21, 18.80s/it]

Какие языки программирования популярны в 2025 году?
content='Прогнозировать точный список самых популярных языков программирования на конкретную дату в будущем затруднительно, но можно предположить некоторые тренды и направления развития на основе текущих тенденций.\n\n1. **Python** - язык продолжает оставаться популярен благодаря своей простоте использования и широкому применению в таких областях как анализ данных, машинное обучение, веб-разработка и автоматизация задач.\n2. **JavaScript** - поскольку он является основным языком для фронтенд разработки веб-сайтов и все больше используется на серверной стороне благодаря Node.js, ожидается что JavaScript будет продолжать сохранять свою популярность.\n3. **Java** - язык широко используют в корпоративных приложениях и Android разработке, поэтому он остается одним из самых востребованных языков программирования.\n4. **C# и C++** - эти языки продолжают оставаться популярными для игр разработки (Unity использует C#, а многие игры разрабатыва

...

Что такое обработка естественного языка?
content='Обработка естественного языка (ОЕЯ или NLP от англ. Natural Language Processing) — это область искусственного интеллекта и компьютерной лингвистики, которая занимается разработкой алгоритмов и методов для общения между человеком и компьютером в естественном языке. ОЕЯ сочетает в себе знания из таких областей как лингвистика, статистика, машинное обучение и искусственный интеллект.\n\nОсновные задачи в области ОЕЯ включают:\n\n1. **Анализ настроения:** определение эмоционального состояния автора текста.\n2. **Перевод между языками:** автоматический перевод с одного языка на другой.\n3. **Извлечение информации:** извлечение структурированной информации из неструктурированных текстовых данных.\n4. **Разбор синтаксиса:** анализ грамматической структуры предложений.\n5. **Генерация текста:** создание осмысленного текста в ответ на запрос пользователя.\n\nNLP используется во многих приложениях, таких как чат-боты, системы автоматического пере

 42%|████████████████████████████████████████▎                                                       | 21/50 [05:52<08:09, 16.86s/it]

Что такое глубокая нейронная сеть?


 42%|████████████████████████████████████████▎                                                       | 21/50 [05:55<08:11, 16.94s/it]


KeyboardInterrupt: 